# Library

In [48]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import FinanceDataReader as fdr

import datetime as dt

import numpy as np

from tqdm import tqdm #진행바
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

# GPU
import tensorflow as tf

#https://www.tensorflow.org/guide/gpu#allowing_gpu_memory_growth
#프로세스의 요구량만큼 메모리 사용 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
#지정해서 쓰고 싶으면 model 쓸 때는 with 구문 안에 넣기

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# with open

In [50]:
data = pd.read_csv('stock_2018.csv')
data.columns

# min max idx
max_idx = data['High'].argmax()
min_idx = data['Low'].argmin()

# min max value
maxv = max(data['High'])
minv = min(data['Low'])
maxv
minv

np.where(data['Trading_Value']>= 1000_0000_0000)

Index(['Index', 'Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Trading_Value', 'Change', 'Next_Change', 'MA5', 'MA20', 'MA60',
       'MA120', 'MFI', 'ADI', 'OBV', 'CMF', 'FI', 'EOM_EMV', 'VPT', 'NVI',
       'VMAP', 'ATR', 'BHB', 'BLB', 'KCH', 'KCL', 'KCM', 'DCH', 'DCL', 'DCM',
       'UI', 'SMA', 'EMA', 'WMA', 'MACD', 'ADX', '-VI', '+VI', 'TRIX', 'MI',
       'CCI', 'DPO', 'KST', 'Ichimoku', 'Parabolic_SAR', 'STC', 'RSI', 'SRSI',
       'TSI', 'UO', 'SR', 'WR', 'AO', 'KAMA', 'ROC', 'PPO', 'PVO'],
      dtype='object')

1709000

143

(array([     0,      1,      2, ..., 379718, 379720, 379721]),)

In [21]:
# import multiprocessing
# from multiprocessing import Pool

# multiprocessing.cpu_count() # pc에 128개의 cpu core 존재

# ### 멀티 프로세싱 ###
# p = Pool(20)

128

# try2

In [73]:
# %%time

cd_dt_dic = {}
x_df = []
y_df = []
minmax_idx = []

f = open('stock_2018.csv')
lines = f.readlines()
for i,line in tqdm(enumerate(lines)):
    line = line.strip()
    lst = np.array(line.split(','))
    lst = lst[1:]
    if i == 0 : # colname에서 원하는 거 (idx) 뽑아서 변수에 저장. ; i==0일 때 lst == 위에서의 data.columns.values
        
        # 마지막 식별변수 'Date'열 idx # 1
        id_idx = np.where(lst == 'Date')[0][0]

        # MinMax에 쓸 변수들 idx
        for mm in ['Open', 'High', 'Low', 'Close', 'MA5', 'MA20', 'MA60', 'MA120', 'VMAP', 'BHB', 'BLB', 'KCH', 'KCL', 'KCM', 'DCH', 'DCL', 'DCM','SMA', 'EMA', 'WMA', 'Ichimoku', 'Parabolic_SAR', 'KAMA','MACD']:
            minmax_idx.append(np.where(lst == mm)[0][0])
        minmax_idx = sorted(minmax_idx)

        # 목적변수 열 idx # 9
        목적_idx = np.where(lst == 'Next_Change')[0][0]

        x0 = np.append(lst[0:목적_idx], lst[목적_idx+1:])
        y0 = np.append(lst[0:2], lst[목적_idx])
        
        # 조건에 쓸 TV idx # 7
        TV_idx = np.where(x0 == 'Trading_Value')[0][0]
        
    else:
        # 식별변수 먼저 추가해놓고 시작
        x1 = lst[0:id_idx+1] # ndarray
        y1 = lst[0:id_idx+1]

        # 식별변수 열이 아닌 곳부터 뽑고 enumerate는 +2해서 이용 하기로
        for k,j in enumerate(lst[id_idx+1:]):
    
            # TV면 형변환만해서 그냥 집어넣기(Trading_Value 열은 안바꿈., 이유는..? 조건에 사용하기 위한 것 뿐인가?)
            if k+2 == TV_idx:
                x1 = np.append(x1, float(j))
                
            elif k+2 in minmax_idx:
                tt = (float(j)-minv) / (maxv - minv)
                x1 = np.append(x1, tt)

            # 목적변수 열이 아니면 (x먼저 만들어줄 거임.)
            elif k+2 != 목적_idx:
                tt = (float(j)-minv) / (maxv - minv)
                x1 = np.append(x1, tt)

            # 목적변수면 목적변수는 minmax 안함.
            else:
                y1 = np.append(y1,float(j))
#             break # for check 1 loop

        x_df.append(x1)
        y_df.append(y1)
#         print(x_df)
f.close()

# id_idx
# minmax_idx
# 목적_idx
# x0
# y0
# TV_idx

0it [00:00, ?it/s]

In [76]:
x_df

[array(['050120', '2018-01-02', '0.005914479678521959',
        '0.006967815329193724', '0.005855961031262417',
        '0.006821518711044868', '15.2655406508561', '307823874200.0',
        '-8.359644428995521e-05', '0.005831383199413409',
        '0.0053635266145733666', '0.004961310396364354',
        '0.00501939015376945', '-3.169855640349075e-05',
        '-7.6238532205229355', '175.77663022710502',
        '-8.368535506481819e-05', '10852.097440512918',
        '2.2162281571834272', '3.7671242285989823',
        '0.00016436951716849334', '0.006234869272267954',
        '0.000544246826972649', '0.00671132809825515',
        '0.004015701723432681', '0.005990436882664845',
        '0.004758636913445654', '0.005374545675852339',
        '0.0076115204490486916', '0.004609513844634162',
        '0.006125146808656313', '-7.684288386915933e-05',
        '0.005662849495305927', '0.0057476254595908265',
        '0.006003545059650984', '0.00025883090276131935',
        '-5.844070042139278e-0

# 악...... 버려.... (try 1)

In [72]:
'''
%%time

cd_dt_dic = {}
x_df = []
y_df = []
minmax_idx = []

f = open('stock_2018.csv')
lines = f.readlines()
for i,line in tqdm(enumerate(lines)):
    line = line.strip()
    lst = np.array(line.split(','))
    lst = lst[1:]

    if i == 0 : # colname에서 원하는 거 (Next_Change idx) 뽑아서 변수에 저장. ; i==0일 때 lst == 위에서의 data.columns.values
        
        # 마지막 식별변수 'Date'열 idx
        id_idx = np.where(lst == 'Date')[0][0]

        # MinMax에 쓸 변수들 idx
        for mm in ['Open', 'High', 'Low', 'Close', 'MA5', 'MA20', 'MA60', 'MA120', 'VMAP', 'BHB', 'BLB', 'KCH', 'KCL', 'KCM', 'DCH', 'DCL', 'DCM','SMA', 'EMA', 'WMA', 'Ichimoku', 'Parabolic_SAR', 'KAMA','MACD']:
            minmax_idx.append(np.where(lst == mm)[0][0])

        # 목적변수 열 idx
        목적_idx = np.where(lst == 'Next_Change')[0][0]

        x0 = np.append(lst[0:목적_idx], lst[목적_idx+1:])
        y0 = np.append(lst[0:2], lst[목적_idx])
        
        # 조건에 쓸 TV idx
        TV_idx = np.where(x0 == 'Trading_Value')[0][0]

    else:
        
        # 먼저 TV열 >= 1000_0000_0000인 애들인지 확인부터 하고 시작! 맞으면 다음단계로 넘어가기로!!
        # 앗....... 다른 날짜들의 데이터도 필요하니까 그냥 처음처럼 1000억 조건은 나중에 걸어야할듯.
               
        # 식별변수 먼저 추가해놓고 시작
        x1 = lst[0:id_idx+1] # ndarray
        y1 = lst[0:id_idx+1]
        
#         print(lst[10])

#         for k,j in tqdm(enumerate(lst)):
        for k,j in enumerate(lst): #아.......loop name을 똑같이 쓰니까 이런 일이 일어날 수 밖에....

            # 식별변수 열이 아니면
#             while k > id_idx:
            if k > id_idx:
                
                # TV면 형변환만해서 그냥 집어넣기(Trading_Value 열은 안바꿈., 이유는..? 조건에 사용하기 위한 것 뿐인가?)
                if k == TV_idx: #idx ==7
#                     print('TV_idx', k)
                    x1 = np.append(x1, float(j))
                    
            ##############################################################
#                     if float(j) >= 1000_0000_0000:
#                         # 코드랑 날짜 같이 부르기
#                         tmp_cd = x1[0]
#                         tmp_dt = x1[1]

#                         # 사전에 key가 이미 있으면 해당 키의 val list에 값 append하기
#                         if tmp_cd not in cd_dt_dic: # cd_dt_dic = {} 위치가 아예 밖에 있어야지... 안에 있음 초기화되니까....
#                             globals()[f'data_{tmp_cd}'] = [tmp_dt]
#                             cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']

#                         else:
#                             globals()[f'data_{tmp_cd}'].append(tmp_dt)
#                             cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']
            ################################################################
                
                # minmax를 해야하는 열이면 minmax해서 x1에 append
                elif k in minmax_idx:
#                     print(k)
                    tt = (float(j)-minv) / (maxv - minv)
                    x1 = np.append(x1, tt)
                
                # 목적변수 열이 아니면 (x먼저 만들어줄 거임.)
                elif k != 목적_idx:
#                     print(k)
                    tt = (float(j)-minv) / (maxv - minv)
                    x1 = np.append(x1, tt)
                
                # 목적변수면 목적변수는 minmax 안함.
                else:
#                     print('목적idx', k)
                    y1 = np.append(y1,float(j))
#                 break# #while과 세트
            else:
                pass
            
# #         print(x1)
# #         print(y1)
        x_df.append(x1)
        y_df.append(y1)
        print(x_df)
# #         print(y_df)
# #         break
################################################## TV조건하는걸 제외했다면 여기까지 4분 걸리면 정상.
        
#         if float(x1[TV_idx]) >= 1000_0000_0000:
#             # 코드랑 날짜 같이 부르기
#             tmp_cd = x1[0]
#             tmp_dt = x1[1]
            
#             # 사전에 key가 이미 있으면 해당 키의 val list에 값 append하기
#             if tmp_cd not in cd_dt_dic: # cd_dt_dic = {} 위치가 아예 밖에 있어야지... 안에 있음 초기화되니까....
#                 globals()[f'data_{tmp_cd}'] = [tmp_dt]
#                 cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']

#             else:
#                 globals()[f'data_{tmp_cd}'].append(tmp_dt)
#                 cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']
            
#         break#
'''

"\n%%time\n\ncd_dt_dic = {}\nx_df = []\ny_df = []\nminmax_idx = []\n\nf = open('stock_2018.csv')\nlines = f.readlines()\nfor i,line in tqdm(enumerate(lines)):\n    line = line.strip()\n    lst = np.array(line.split(','))\n    lst = lst[1:]\n\n    if i == 0 : # colname에서 원하는 거 (Next_Change idx) 뽑아서 변수에 저장. ; i==0일 때 lst == 위에서의 data.columns.values\n        \n        # 마지막 식별변수 'Date'열 idx\n        id_idx = np.where(lst == 'Date')[0][0]\n\n        # MinMax에 쓸 변수들 idx\n        for mm in ['Open', 'High', 'Low', 'Close', 'MA5', 'MA20', 'MA60', 'MA120', 'VMAP', 'BHB', 'BLB', 'KCH', 'KCL', 'KCM', 'DCH', 'DCL', 'DCM','SMA', 'EMA', 'WMA', 'Ichimoku', 'Parabolic_SAR', 'KAMA','MACD']:\n            minmax_idx.append(np.where(lst == mm)[0][0])\n\n        # 목적변수 열 idx\n        목적_idx = np.where(lst == 'Next_Change')[0][0]\n\n        x0 = np.append(lst[0:목적_idx], lst[목적_idx+1:])\n        y0 = np.append(lst[0:2], lst[목적_idx])\n        \n        # 조건에 쓸 TV idx\n        TV_idx = np.where(x0 == 'Trading_

In [77]:
%%time
###################################################################
x_df = pd.DataFrame(x_df, columns = x0)
y_df = pd.DataFrame(y_df, columns = y0)
###################################################################

CPU times: user 16.7 s, sys: 399 ms, total: 17.1 s
Wall time: 17.1 s


# to csv

In [78]:
# x_df.to_csv('repreprocessing_x_df.csv', index = False)
# y_df.to_csv('repreprocessing_y_df.csv', index = False)

In [80]:
pd.read_csv('repreprocessing_x_df.csv')
pd.read_csv('repreprocessing_y_df.csv')

,Code,Date,Open,High,Low,Close,Volume,Trading_Value,Change,MA5,...,SRSI,TSI,UO,SR,WR,AO,KAMA,ROC,PPO,PVO
0,50120,2018-01-02,0.005914,0.006968,0.005856,0.006822,15.265541,3.078239e+11,-0.000084,0.005831,...,-0.000083,-0.000071,-0.000057,-0.000041,-0.000099,0.000653,0.005708,-0.000057,-0.000080,-0.000059
1,50120,2018-01-03,0.006909,0.007202,0.006295,0.006792,11.973109,2.404106e+11,-0.000084,0.006083,...,-0.000083,-0.000069,-0.000058,-0.000041,-0.000100,0.000756,0.005807,-0.000058,-0.000079,-0.000062
2,50120,2018-01-04,0.006851,0.008197,0.006704,0.007290,35.499583,7.643646e+11,-0.000084,0.006489,...,-0.000083,-0.000067,-0.000057,-0.000040,-0.000098,0.001080,0.005898,-0.000053,-0.000079,-0.000059
3,50120,2018-01-05,0.007407,0.007641,0.006939,0.007056,8.154641,1.700101e+11,-0.000084,0.006781,...,-0.000083,-0.000067,-0.000056,-0.000044,-0.000102,0.001364,0.005949,-0.000054,-0.000078,-0.000064
4,50120,2018-01-08,0.007202,0.007758,0.007143,0.007436,9.638936,2.116614e+11,-0.000084,0.007079,...,-0.000083,-0.000066,-0.000055,-0.000038,-0.000096,0.001551,0.005979,-0.000068,-0.000078,-0.000068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380584,540,2018-12-21,0.002538,0.002538,0.002450,0.002538,0.063798,4.890547e+08,-0.000084,0.002592,...,-0.000084,-0.000112,-0.000060,-0.000071,-0.000130,-0.000296,0.002604,-0.000090,-0.000086,-0.000082
380585,540,2018-12-24,0.002538,0.002567,0.002515,0.002561,0.021552,1.671180e+08,-0.000084,0.002570,...,-0.000084,-0.000112,-0.000056,-0.000068,-0.000126,-0.000314,0.002597,-0.000089,-0.000086,-0.000082
380586,540,2018-12-26,0.002558,0.002558,0.002418,0.002532,0.016739,1.284991e+08,-0.000084,0.002557,...,-0.000084,-0.000113,-0.000052,-0.000068,-0.000126,-0.000331,0.002585,-0.000088,-0.000086,-0.000082
380587,540,2018-12-27,0.002535,0.002550,0.002497,0.002532,0.011981,9.215799e+07,-0.000084,0.002540,...,-0.000084,-0.000113,-0.000052,-0.000067,-0.000126,-0.000336,0.002572,-0.000088,-0.000086,-0.000084


,Code,Date,Next_Change
0,50120,2018-01-02,-0.004237
1,50120,2018-01-03,0.072340
2,50120,2018-01-04,-0.031746
3,50120,2018-01-05,0.053279
4,50120,2018-01-08,-0.089494
...,...,...,...
380584,540,2018-12-21,0.008929
380585,540,2018-12-24,-0.011062
380586,540,2018-12-26,0.000000
380587,540,2018-12-27,0.052573


In [81]:
%%time

# data load
data = pd.read_csv('stock_2018.csv')
data = data.drop('Index', axis=1)


# col names
col_arr = data.columns.values

# 설명 idx 필요 없음, 식별변수 필요해서 0번째 열부터 써야함.
목적_idx = np.where(col_arr == 'Next_Change')[0][0] # 목적변수 열 idx

# 사실 굳이 나눌 필요는 없긴 함. 처음에 코드 쓸 때 나눠서 썼으니 계속 나눠서 쓰는것..
x = pd.concat([data.iloc[:, 0:목적_idx], data.iloc[:,목적_idx+1:]], axis = 1)
y = pd.concat([data.iloc[:, 0:2], data[['Next_Change']]], axis = 1)


# 여기부터 with open문 안으로 들어가면 될 것 같음.
# (x[['Trading_Value']] >= 1000_0000_0000).sum() #2682
천억 = x[x['Trading_Value'] >= 1000_0000_0000]


# 코드랑 날짜 같이 부르기
cd_dt_dic = {}
for i in range(len(천억[['Code','Date']])):
    
    tmp = 천억[['Code','Date']].iloc[[i]]
    tmp_cd = tmp['Code'].values[0]
    tmp_dt = tmp['Date'].values[0]
    
    # 사전에 key가 이미 있으면 해당 키의 val list에 값 append하기 # 뭔가 함수가 있을 줄 알았는데 없었다. update는 key에 매치된 val값을 새로 바꾸는거..
    if tmp_cd not in cd_dt_dic:
        globals()[f'data_{tmp_cd}'] = [tmp_dt]
        cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']
    else:
        globals()[f'data_{tmp_cd}'].append(tmp_dt)
        cd_dt_dic[tmp_cd] = globals()[f'data_{tmp_cd}']
# cd_dt_dic


CPU times: user 2.39 s, sys: 104 ms, total: 2.5 s
Wall time: 2.46 s


In [110]:
# cd_dt_dic
y_df.iloc[1]['Next_Change']
y_df
# x_df

'0.0723404'

,Code,Date,Next_Change
0,050120,2018-01-02,-0.00423729
1,050120,2018-01-03,0.0723404
2,050120,2018-01-04,-0.031746
3,050120,2018-01-05,0.0532787
4,050120,2018-01-08,-0.0894942
...,...,...,...
380584,000540,2018-12-21,0.00892857
380585,000540,2018-12-24,-0.0110619
380586,000540,2018-12-26,0.0
380587,000540,2018-12-27,0.0525727


In [116]:
# %%time
dday_alldf = pd.DataFrame()

# need_date가 아닌 cd_dt_dic에서 고르기
for k,(key, val) in tqdm(enumerate(cd_dt_dic.items())):
#     k,key

    # 종목코드는 max 6자리임.
    if len(str(key)) !=6:
        new_key = '0'*(6-len(str(key)))+str(key)
    
#     key인 동안에
    for i in val: # key가 문제인 것. 그냥 key로 쓰이는 부분을 new_key로 싹다 바꿔줄 것.
        if np.where(x_df[x_df['Code']== new_key]['Date'].values == i)[0][0] >= 9 :
            idx = x_df[x_df['Code']== new_key][x_df['Date']==i].index
            
            #dday_df : 한 date에 대한 data 한 줄
            dday_df = pd.DataFrame([x_df.iloc[:,0:2].loc[idx.values[0]]])
            dday_df.rename(index = {idx.values[0] : i}, inplace = True) # dday_df의 code와 같은 code인 df(x)중에서 고르기

            for j in range(9, -1, -1):
                tmp = x_df.iloc[idx-j].iloc[:,2:]
                tmp.columns = [f'D-{j}_{ori_col}' for ori_col in x_df.iloc[:,2:].columns]
                tmp = tmp.rename(index = {idx.values[0]-j : i})
        
                dday_df = pd.concat([dday_df, tmp], axis = 1) # dday_df 옆으로 concat
            
            # dday_df 옆에 같은 idx의 y 붙여주기
            dday_df = pd.concat([dday_df, y_df.iloc[idx]['Next_Change'].rename(index = {idx.values[0]:i})], axis = 1)
            
            # dday_alldf: data 아래로 이어붙이기
            dday_alldf = pd.concat([dday_alldf, dday_df])
    
# f.close()

0it [00:00, ?it/s]

(0, 50120)

(1, 67290)

(2, 36120)

(3, 78130)

...

(236, 180640)

(237, 5440)

(238, 64350)

(239, 5380)

(240, 11760)

In [117]:
# 에러메세지: _append_dispatcher() missing 1 required positional argument: 'values'
# np.append는 1개의 argument를 더 필요로 한다.는 뜻...
# x1#.shape
# y1#.shape
# '####################################################'
# x_df
# y_df
# '####################################################'
# cd_dt_dic

x1.shape

x0
high_idx
low_idx

In [119]:
dday_alldf
# dday_alldf.to_csv('minmax_stock_2018_10days.csv', index = False)

,Code,Date,D-9_Open,D-9_High,D-9_Low,D-9_Close,D-9_Volume,D-9_Trading_Value,D-9_Change,D-9_MA5,...,D-0_TSI,D-0_UO,D-0_SR,D-0_WR,D-0_AO,D-0_KAMA,D-0_ROC,D-0_PPO,D-0_PVO,Next_Change
2018-01-15,050120,2018-01-15,0.005914479678521959,0.006967815329193724,0.005855961031262417,0.006821518711044868,15.2655406508561,307823874200.0,-8.359644428995521e-05,0.005831383199413409,...,-6.65225352384664e-05,-5.712666419717975e-05,-4.6274907730722935e-05,-0.00010479355499026543,0.002049837991125062,0.006855927675633478,-4.349416013159673e-05,-7.646947638099619e-05,-6.094600074786831e-05,-0.0681818
2018-01-16,050120,2018-01-16,0.0069092966819341815,0.0072018899182318945,0.006294850885708986,0.006792259387415097,11.973108926024823,240410569500.0,-8.368414518593422e-05,0.006083013382629442,...,-6.725612500051204e-05,-5.6973403860007006e-05,-5.251931554249419e-05,-0.00011103796280203668,0.002245793533338366,0.006907892234399953,-5.571806183899531e-05,-7.648960679565347e-05,-6.575886689172938e-05,-0.188153
2018-01-17,050120,2018-01-17,0.006850778034674639,0.008196706921644117,0.006704481416525783,0.007289667889121208,35.49958305463828,764364560400.0,-8.363933295764362e-05,0.006489132794610667,...,-7.06238731502987e-05,-5.9668304603603454e-05,-6.857624716404005e-05,-0.00012709489442358255,0.002074281230085373,0.00690707297333832,-7.601174352213205e-05,-7.757038769188996e-05,-6.981274618063418e-05,0.0472103
2018-01-18,050120,2018-01-18,0.007406705183640293,0.007640779772678463,0.006938556005563953,0.007055593300083038,8.154640792061594,170010147600.0,-8.370024291090477e-05,0.006780555657963188,...,-7.26336960904277e-05,-5.946471822978752e-05,-6.678586915113436e-05,-0.00012530451641067685,0.0016646506992685755,0.006908067790341731,-8.169798292074762e-05,-7.828391140979027e-05,-7.348935575065672e-05,0.0122951
2018-01-19,050120,2018-01-19,0.0072018899182318945,0.007757817067197548,0.007143371270972352,0.007435964507270064,9.638936435289787,211661434950.0,-8.365048760662829e-05,0.007079000758986855,...,-7.400367614142085e-05,-6.404854238827474e-05,-6.905200376626013e-05,-0.00012757065102580263,0.001459659877918398,0.006909238163286922,-8.069348108121394e-05,-7.884762153884146e-05,-7.625143590130713e-05,0.137652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-15,011760,2018-05-15,0.012439308847960948,0.014019312323968594,0.012088196964403691,0.013024495320556372,0.8712027981276373,33351449600.0,-8.362586541764465e-05,0.012105752558581555,...,-5.051786076892332e-05,-4.3874297264194726e-05,-3.415329661873405e-05,-9.267194387827653e-05,0.003934741174949103,0.01613915032094552,-3.982937132832063e-05,-7.606768734891216e-05,-4.361096335152678e-05,-0.0974212
2018-05-17,011760,2018-05-17,0.01366820044041134,0.013873015705819738,0.012761161407888431,0.012995235996926601,0.2862632742236477,10936436100.0,-8.371961703056487e-05,0.01281382819042202,...,-5.1765419809849505e-05,-4.622393798895987e-05,-3.3827172197556615e-05,-9.23458194570991e-05,0.005966544063216111,0.01718961855790157,-3.503534818887713e-05,-7.461782934440975e-05,-4.440523695078055e-05,0.0280374
2018-06-07,011760,2018-06-07,0.0242893349180183,0.02443563153616716,0.021568217820449577,0.02262155347112134,0.7637344728084328,50643932800.0,-8.371575409762197e-05,0.022849776195433557,...,-6.639192161778312e-05,-5.4523345136544486e-05,-4.2348247980960366e-05,-0.00010086689524050286,0.00581973798860876,0.021333441007644293,-7.676581481071852e-05,-7.605727102969997e-05,-7.324954633418712e-05,-0.00116144
2018-06-11,011760,2018-06-11,0.023616370474533563,0.024084519652609902,0.02335303656186562,0.024084519652609902,0.4168739689745836,29427117300.0,-8.350684580394966e-05,0.022516219906054163,...,-6.503545937430692e-05,-5.21449717559749e-05,-2.760131479696663e-05,-8.611994450091494e-05,0.006341490247574842,0.022793422738122616,-7.146660018948338e-05,-7.55435943440557e-05,-7.232085540217818e-05,-0.0381526


In [120]:
pd.read_csv('minmax_stock_2018_10days.csv')

,Code,Date,D-9_Open,D-9_High,D-9_Low,D-9_Close,D-9_Volume,D-9_Trading_Value,D-9_Change,D-9_MA5,...,D-0_TSI,D-0_UO,D-0_SR,D-0_WR,D-0_AO,D-0_KAMA,D-0_ROC,D-0_PPO,D-0_PVO,Next_Change
0,50120,2018-01-15,0.005914,0.006968,0.005856,0.006822,15.265541,3.078239e+11,-0.000084,0.005831,...,-0.000067,-0.000057,-0.000046,-0.000105,0.002050,0.006856,-0.000043,-0.000076,-0.000061,-0.068182
1,50120,2018-01-16,0.006909,0.007202,0.006295,0.006792,11.973109,2.404106e+11,-0.000084,0.006083,...,-0.000067,-0.000057,-0.000053,-0.000111,0.002246,0.006908,-0.000056,-0.000076,-0.000066,-0.188153
2,50120,2018-01-17,0.006851,0.008197,0.006704,0.007290,35.499583,7.643646e+11,-0.000084,0.006489,...,-0.000071,-0.000060,-0.000069,-0.000127,0.002074,0.006907,-0.000076,-0.000078,-0.000070,0.047210
3,50120,2018-01-18,0.007407,0.007641,0.006939,0.007056,8.154641,1.700101e+11,-0.000084,0.006781,...,-0.000073,-0.000059,-0.000067,-0.000125,0.001665,0.006908,-0.000082,-0.000078,-0.000073,0.012295
4,50120,2018-01-19,0.007202,0.007758,0.007143,0.007436,9.638936,2.116614e+11,-0.000084,0.007079,...,-0.000074,-0.000064,-0.000069,-0.000128,0.001460,0.006909,-0.000081,-0.000079,-0.000076,0.137652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2532,11760,2018-05-15,0.012439,0.014019,0.012088,0.013024,0.871203,3.335145e+10,-0.000084,0.012106,...,-0.000051,-0.000044,-0.000034,-0.000093,0.003935,0.016139,-0.000040,-0.000076,-0.000044,-0.097421
2533,11760,2018-05-17,0.013668,0.013873,0.012761,0.012995,0.286263,1.093644e+10,-0.000084,0.012814,...,-0.000052,-0.000046,-0.000034,-0.000092,0.005967,0.017190,-0.000035,-0.000075,-0.000044,0.028037
2534,11760,2018-06-07,0.024289,0.024436,0.021568,0.022622,0.763734,5.064393e+10,-0.000084,0.022850,...,-0.000066,-0.000055,-0.000042,-0.000101,0.005820,0.021333,-0.000077,-0.000076,-0.000073,-0.001161
2535,11760,2018-06-11,0.023616,0.024085,0.023353,0.024085,0.416874,2.942712e+10,-0.000084,0.022516,...,-0.000065,-0.000052,-0.000028,-0.000086,0.006341,0.022793,-0.000071,-0.000076,-0.000072,-0.038153
